## Lista 1
### Zadanie 1
* Wyznaczymy *macheps* iteracyjnie poprzez dzielenie liczby 1.0 (w różnych reprezentacjach) przez 2, tak długo jak suma 1 + *macheps* będzie > 1.

In [1]:
function print_macheps(breed)
    macheps = breed(1.0) 
    previous_macheps = macheps
    
    while 1 + macheps != 1 #adding and dividing any Float type T with Int returns T.
        previous_macheps = macheps
        macheps /= 2 
        #println(macheps)
        #println(bitstring(macheps)
    end
    println(breed, ": ", previous_macheps)
end

print_macheps (generic function with 1 method)

In [2]:
println("by me: ")
half = Float16
print_macheps(half)
single = Float32
print_macheps(single)
double = Float64
print_macheps(double)
#####################
println("by Julia: ")
println(half, ": ", eps(half))
println(single, ": ", eps(single))
println(double, ": ", eps(double))

by me: 
Float16: 0.000977
Float32: 1.1920929e-7
Float64: 2.220446049250313e-16
by Julia: 
Float16: 0.000977
Float32: 1.1920929e-7
Float64: 2.220446049250313e-16


Wyniki zgadzają się. Zobaczmy jeszcze *float.h* w C. *float.h* oferuje implemntacje **single** i **double**, nie ma **half**, ale ma za to **long double**. Kod w C oraz wynik:

    #include <stdio.h>
    #include <float.h>

    int main () {

       printf("The macheps value of Float32 = %.10e\n", FLT_EPSILON);
   
       printf("The macheps value of Float64 = %.10e\n", DBL_EPSILON);
   
    }

The macheps value of Float32 = 1.1920928955e-07

The macheps value of Float64 = 2.2204460493e-16


* Wyznaczymy *ete* iteracyjnie poprzez dzielenie liczby 1.0 (w różnych reprezentacjach) przez 2, tak długo jak będzie więszka od 0.0.

In [3]:
function print_eta(breed)
    eta = breed(1.0) 
    previous_eta = eta
    
    while eta > breed(0.0)
        previous_eta = eta
        eta /= 2 
        #println(eta)
        #println(bitstring(eta)
    end
    println(breed, ": ", previous_eta)
end

print_eta (generic function with 1 method)

In [4]:
println("by me: ")
half = Float16
print_eta(half)
single = Float32
print_eta(single)
double = Float64
print_eta(double)
#####################
println("by Julia: ")
println(half, ": ", nextfloat(half(0.0)))
println(single, ": ", nextfloat(single(0.0)))
println(double, ": ", nextfloat(double(0.0)))

by me: 
Float16: 6.0e-8
Float32: 1.0e-45
Float64: 5.0e-324
by Julia: 
Float16: 6.0e-8
Float32: 1.0e-45
Float64: 5.0e-324


* Odpowiedzi na pytania:

Liczba *macheps* jest najmniejszą liczbą maszynową, większą od ϵ gdzie 1 + ϵ = 1. *Macheps* jest więskzy od ϵ o ostani bit reprezentacji, tym samy *macheps* = 2ϵ.

Liczba *eta* jest górną granicą zbioru liczb większych od 0, które nie mają reprezentacji w zbiorze liczb maszynowych.

Funckje floatmin(Float32), floatmin(Float64) zwracają najmniejszą, znormalizowaną, dodatnią liczbę reprezentowaną przez dany typ zmiennoprzecinkowy (co równa się $MIN_{nor}$). Kiedy nextfloat() zwraca zdenormalizowaną liczbę co pokazuje poniższy kod. W dokumetnacji IEEE-754 możemy przeczytać:
>Underflow:
If the exponent has minimum value (all zero), special rules for denormalized values are followed. The exponent value is set to 2-126 and the "invisible" leading bit for the mantissa is no longer used.
The range of the mantissa is now [0:1).

In [5]:
println(floatmin(Float16))
println(bitstring(floatmin(Float16)))

6.104e-5
0000010000000000


In [6]:
println(nextfloat(Float16(0.0)))
println(bitstring(nextfloat(Float16(0.0))))

6.0e-8
0000000000000001


* Wyznaczymy *max* iteracyjnie poprzez mnożenie x = 1.0 (w różnych reprezentacjach) przez 2, tak długo aż nie otrzymamy *inf*, wtedy potrzebujemy dodać (zapełnić) pozostałe bity mantysy x, najszybciej osiągniemy to odejmując od x eps(x) co da nam największa liczbę mniejszą od x, a następnie dodając ją do x. Musimy wykonać tę operację ponieważ ciągłe mnożenie przez jeden zapełnia wszytskie bity cechy co powoduje, że x ląduje w special case, mianowicie inf.

In [7]:
function print_max(breed)
    max = breed(1.0) 
    previous_max = max
    
    while isinf(max) == false
        previous_max = max
        max *= 2
        #println(max)
        #println(bitstring(max)
    end

    max = previous_max + (previous_max - eps(previous_max)) # very important bracket
    println(breed, ": ", max)
end

print_max (generic function with 1 method)

In [8]:
println("by me: ")
half = Float16
print_max(half)
single = Float32
print_max(single)
double = Float64
print_max(double)
#####################
println("by Julia: ")
println(half, ": ", floatmax(half))
println(single, ": ", floatmax(single))
println(double, ": ", floatmax(double))

by me: 
Float16: 6.55e4
Float32: 3.4028235e38
Float64: 1.7976931348623157e308
by Julia: 
Float16: 6.55e4
Float32: 3.4028235e38
Float64: 1.7976931348623157e308


In [9]:
println(bitstring(Float16(3.277e4))) # x
println(bitstring((Float16(3.277e4)) - eps(Float16(3.277e4)))) # x - eps(x)
println(bitstring((Float16(3.277e4) + ((Float16(3.277e4)) - eps(Float16(3.277e4)))))) # x + (x - eps(x)) = max
println(bitstring(floatmax(Float16))) # max

0111100000000000
0111011111111110
0111101111111111
0111101111111111


### Zadanie 2
* Będziemy obliczać wyrażenie $3(\frac{4}{3} - 1) - 1$ dla wszystkich typów zmiennoprzecinkowych.

In [10]:
println("by Kahan: ")
println(Float16(3) * ( Float16(4/3) - Float16(1) ) - Float16(1))
println(Float32(3) * ( Float32(4/3) - Float32(1) ) - Float32(1))
println(Float64(3) * ( Float64(4/3) - Float64(1) ) - Float64(1))
println("by Julia: ")
println(eps(Float16))
println(eps(Float32))
println(eps(Float64))

by Kahan: 
-0.000977
1.1920929e-7
-2.220446049250313e-16
by Julia: 
0.000977
1.1920929e-7
2.220446049250313e-16


Z dokładnością do wartości bezwględnej twierdzenie Kahana jest prawdziwe.

### Zadanie 3
W IEE-754 (double) reprezentacje bitowe:

In [16]:
println("1 bitowo: ", bitstring(1.0))
println("2 bitowo: ", bitstring(2.0))

liczba 1: 0011111111110000000000000000000000000000000000000000000000000000
liczba 2: 0100000000000000000000000000000000000000000000000000000000000000


Mamy 52 bity mantysy co oznacza, że za ich pomocą możemy zapisać $2^{52}$ różnych liczb z przedziału [1,2).
Przy stałej cesze kolejną liczbę otrzymujemy poprzez dodanie bita na samym końcu (czyli 52-ego bita), co odpowiada dodaniu $2^{-52}$, co za tym idzie liczby maszynowe w przedziale [1,2] są rozmieszcozne równomiernie.

In [22]:
println("1/2 bitowo: ", bitstring(0.5))
println("1   bitowo: ", bitstring(1.0))
println("2   bitowo: ", bitstring(2.0))
println("4   bitowo: ", bitstring(4.0))

1/2 bitowo: 0011111111100000000000000000000000000000000000000000000000000000
1   bitowo: 0011111111110000000000000000000000000000000000000000000000000000
2   bitowo: 0100000000000000000000000000000000000000000000000000000000000000
4   bitowo: 0100000000010000000000000000000000000000000000000000000000000000


Dla każdego przedziału $[2^{c}, 2^{c+1}]$ gdzie $c\in{-1022, -1021, ... ,1023}$ rozkład kolejncyh liczb maszynowych jest stały, ponieważ pomiędzy kolejnymi liczbami różnica w wartosci mantysy jest stała, natomiast poprzez sposób liczenia wartości liczby w IEEE-754, to jest $s* 2^{c} * m_t$ róznice w wartosćiach liczb maszynowych rośnie wraz ze wzrostem cechy. 

### Zadanie 4
* Postaramy się rozwiązać a) i b) za jednym razem. 

In [26]:
x = Float64(1.0)
while Float64(x * Float64(Float64(1.0)/x)) == Float64(1.0) # Float64 in case this will be run on a computer with 32 bit architecture
    x = nextfloat(x)
end

println(x)

1.000000057228997


In [ ]:
# In case if upper cell is calculating too long, result and proof:
1.000000057228997*(1/1.000000057228997)


### Zadanie 5
